In [1]:
from typing import Generator
import utils.llm_utils as llm
import datasets # type: ignore

input_dataset = 'lmsys/chatbot_arena_conversations'
dataset = datasets.load_dataset(input_dataset)['train']
n_rows = len(dataset)
print(str(n_rows))

model_name = 'microsoft/Phi-3-mini-4k-instruct'
tokenizer, model = llm.load_tokenizer_and_model(model_name)

33000


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def add_question_column(dataset: datasets.Dataset) -> datasets.Dataset:
    return dataset.map(
        lambda example: {'question': example['conversation_a'][0]['content']})

# Test:
out_jsonl = "1_dataset_creator/phi3_arena_brief_dataset.jsonl"
with open(out_jsonl, 'w') as f: pass

dataset = dataset.select_columns(['question_id', 'conversation_a']) 
dataset = add_question_column(dataset) 

for big_chunk in llm.process_variable_chunks(dataset, tokenizer, model, 100, 1500, 50):
    big_chunk = big_chunk.select_columns(['question_id', 'question', 'answer'])
    with open(out_jsonl, 'ab') as f:
        big_chunk.to_json(f, lines=True)
    break

Map:   0%|          | 0/33000 [00:00<?, ? examples/s]

AttributeError: module 'utils.llm_utils' has no attribute 'process_variable_chunks'